### Diatom Analysis Pipeline - OTU

#### A notebook  for the diatom analysis pipeline using OTU classification.

### 1. Create BLAST reference database

In [1]:
%%time
!makeblastdb -in diatoms.sequences.FINAL2017.fasta -out diatoms -dbtype nucl



Building a new DB, current time: 06/25/2019 13:29:11
New DB name:   /code/diatoms
New DB title:  diatoms.sequences.FINAL2017.fasta
Sequence type: Nucleotide
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 2701 sequences in 0.189666 seconds.
CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 1.04 s


### 2. Quality Control
* *Cutadapt:* Trim primers from sequences
* *SicklePE:* Trim off bad quality 3' bases
* *Pear:* Merge R1 and R2 reads
* *SickleSE:* Remove post-merging bad quality sequences
* *Histogram Generation:* Plots sequence length against number of sequences
* *QIIME Prep:* Pepare passed QC files for QIIME processing

In [4]:
%%time
!python ./ampliconQC.py --data sequences --forward ATGCGTTGGAGAGARCGTTTC --reverse GATCACCTTCTAATTTACCWACAACTG --threads 8 --histograms --qiime


This is cutadapt 1.9.1 with Python 2.7.16
Command line parameters: -e 0.047619047619 -b ATGCGTTGGAGAGARCGTTTC -b GATCACCTTCTAATTTACCWACAACTG -o /code/sequences/2381.R2.fastq.gz.trimmed.fastq.gz /code/sequences/2381.R2.fastq.gz
Trimming 2 adapters with at most 4.8% errors in single-end mode ...
Finished in 0.86 s (24 us/read; 2.50 M reads/minute).

=== Summary ===

Total reads processed:                  35,894
Reads with adapters:                    31,336 (87.3%)
Reads written (passing filters):        35,894 (100.0%)

Total basepairs processed:    10,357,358 bp
Total written (filtered):      9,511,755 bp (91.8%)

=== Adapter 1 ===

Sequence: ATGCGTTGGAGAGARCGTTTC; Type: variable 5'/3'; Length: 21; Trimmed: 26 times.
5 times, it overlapped the 5' end of a read
21 times, it overlapped the 3' end or was within the read

No. of allowed errors:
0-21 bp: 0

Overview of removed sequences (5')
length	count	expect	max.err	error counts
3	2	560.8	0	2
4	2	140.2	0	2
5	1	35.1	0	1


Overview of rem

length	count	expect	max.err	error counts
3	185	2058.7	0	185
4	6	514.7	0	6
5	4	128.7	0	4
6	1	32.2	0	1
41	2	0.0	1	0 2
51	2	0.0	1	0 2
57	1	0.0	1	1
61	2	0.0	1	0 2
298	2	0.0	1	2
300	2	0.0	1	2

This is cutadapt 1.9.1 with Python 2.7.16
Command line parameters: -e 0.047619047619 -b ATGCGTTGGAGAGARCGTTTC -b GATCACCTTCTAATTTACCWACAACTG -o /code/sequences/3012.R2.fastq.gz.trimmed.fastq.gz /code/sequences/3012.R2.fastq.gz
Trimming 2 adapters with at most 4.8% errors in single-end mode ...
Finished in 1.52 s (21 us/read; 2.83 M reads/minute).

=== Summary ===

Total reads processed:                  71,717
Reads with adapters:                     1,665 (2.3%)
Reads written (passing filters):        71,717 (100.0%)

Total basepairs processed:    20,291,334 bp
Total written (filtered):     20,286,304 bp (100.0%)

=== Adapter 1 ===

Sequence: ATGCGTTGGAGAGARCGTTTC; Type: variable 5'/3'; Length: 21; Trimmed: 1580 times.
4 times, it overlapped the 5' end of a read
1576 times, it overlapped the 3' end o


FastQ paired records kept: 155116 (77558 pairs)
FastQ single records kept: 1328 (from PE1: 1060, from PE2: 268)
FastQ paired records discarded: 388 (194 pairs)
FastQ single records discarded: 1328 (from PE1: 268, from PE2: 1060)



FastQ paired records kept: 165710 (82855 pairs)
FastQ single records kept: 6187 (from PE1: 4317, from PE2: 1870)
FastQ paired records discarded: 2700 (1350 pairs)
FastQ single records discarded: 6187 (from PE1: 1870, from PE2: 4317)



FastQ paired records kept: 161992 (80996 pairs)
FastQ single records kept: 25332 (from PE1: 23557, from PE2: 1775)
FastQ paired records discarded: 5086 (2543 pairs)
FastQ single records discarded: 25332 (from PE1: 1775, from PE2: 23557)



FastQ paired records kept: 147908 (73954 pairs)
FastQ single records kept: 12765 (from PE1: 11930, from PE2: 835)
FastQ paired records discarded: 3036 (1518 pairs)
FastQ single records discarded: 12765 (from PE1: 835, from PE2: 11930)



FastQ paired records kept: 58558 (29279 pairs)
FastQ 


FastQ paired records kept: 121636 (60818 pairs)
FastQ single records kept: 6401 (from PE1: 5844, from PE2: 557)
FastQ paired records discarded: 11504 (5752 pairs)
FastQ single records discarded: 6401 (from PE1: 557, from PE2: 5844)



FastQ paired records kept: 207568 (103784 pairs)
FastQ single records kept: 2841 (from PE1: 2423, from PE2: 418)
FastQ paired records discarded: 418 (209 pairs)
FastQ single records discarded: 2841 (from PE1: 418, from PE2: 2423)



FastQ paired records kept: 170484 (85242 pairs)
FastQ single records kept: 6519 (from PE1: 5418, from PE2: 1101)
FastQ paired records discarded: 1298 (649 pairs)
FastQ single records discarded: 6519 (from PE1: 1101, from PE2: 5418)



FastQ paired records kept: 288006 (144003 pairs)
FastQ single records kept: 4040 (from PE1: 3440, from PE2: 600)
FastQ paired records discarded: 612 (306 pairs)
FastQ single records discarded: 4040 (from PE1: 600, from PE2: 3440)



FastQ paired records kept: 155556 (77778 pairs)
FastQ single re


FastQ paired records kept: 126686 (63343 pairs)
FastQ single records kept: 6047 (from PE1: 5414, from PE2: 633)
FastQ paired records discarded: 1008 (504 pairs)
FastQ single records discarded: 6047 (from PE1: 633, from PE2: 5414)



FastQ paired records kept: 107816 (53908 pairs)
FastQ single records kept: 6688 (from PE1: 5966, from PE2: 722)
FastQ paired records discarded: 1664 (832 pairs)
FastQ single records discarded: 6688 (from PE1: 722, from PE2: 5966)



FastQ paired records kept: 89172 (44586 pairs)
FastQ single records kept: 5814 (from PE1: 5388, from PE2: 426)
FastQ paired records discarded: 1206 (603 pairs)
FastQ single records discarded: 5814 (from PE1: 426, from PE2: 5388)



FastQ paired records kept: 165206 (82603 pairs)
FastQ single records kept: 6927 (from PE1: 5244, from PE2: 1683)
FastQ paired records discarded: 2760 (1380 pairs)
FastQ single records discarded: 6927 (from PE1: 1683, from PE2: 5244)



FastQ paired records kept: 201898 (100949 pairs)
FastQ single rec

...

 ____  _____    _    ____ 
|  _ \| ____|  / \  |  _ \
| |_) |  _|   / _ \ | |_) |
|  __/| |___ / ___ \|  _ <
|_|   |_____/_/   \_\_| \_\

PEAR v0.9.11 [Nov 5, 2017]

Citation - PEAR: a fast and accurate Illumina Paired-End reAd mergeR
Zhang et al (2014) Bioinformatics 30(5): 614-620 | doi:10.1093/bioinformatics/btt593

Forward reads file.................: /code/sequences/2866.sickle.trimmed.R1.fastq.gz
Reverse reads file.................: /code/sequences/2866.sickle.trimmed.R2.fastq.gz
PHRED..............................: 33
Using empirical frequencies........: NO
Statistical method.................: OES
Maximum assembly length............: 999999
Minimum assembly length............: 50
p-value............................: 0.010000
Quality score threshold (trimming).: 0
Minimum read size after trimming...: 1
Maximal ratio of uncalled bases....: 1.000000
Minimum overlap....................: 10
Scoring method.....................: Scaled score
Threads............................: 8

Allo

 ____  _____    _    ____ 
|  _ \| ____|  / \  |  _ \
| |_) |  _|   / _ \ | |_) |
|  __/| |___ / ___ \|  _ <
|_|   |_____/_/   \_\_| \_\

PEAR v0.9.11 [Nov 5, 2017]

Citation - PEAR: a fast and accurate Illumina Paired-End reAd mergeR
Zhang et al (2014) Bioinformatics 30(5): 614-620 | doi:10.1093/bioinformatics/btt593

Forward reads file.................: /code/sequences/3064.sickle.trimmed.R1.fastq.gz
Reverse reads file.................: /code/sequences/3064.sickle.trimmed.R2.fastq.gz
PHRED..............................: 33
Using empirical frequencies........: NO
Statistical method.................: OES
Maximum assembly length............: 999999
Minimum assembly length............: 50
p-value............................: 0.010000
Quality score threshold (trimming).: 0
Minimum read size after trimming...: 1
Maximal ratio of uncalled bases....: 1.000000
Minimum overlap....................: 10
Scoring method.....................: Scaled score
Threads............................: 8

Allo

 ____  _____    _    ____ 
|  _ \| ____|  / \  |  _ \
| |_) |  _|   / _ \ | |_) |
|  __/| |___ / ___ \|  _ <
|_|   |_____/_/   \_\_| \_\

PEAR v0.9.11 [Nov 5, 2017]

Citation - PEAR: a fast and accurate Illumina Paired-End reAd mergeR
Zhang et al (2014) Bioinformatics 30(5): 614-620 | doi:10.1093/bioinformatics/btt593

Forward reads file.................: /code/sequences/2965.sickle.trimmed.R1.fastq.gz
Reverse reads file.................: /code/sequences/2965.sickle.trimmed.R2.fastq.gz
PHRED..............................: 33
Using empirical frequencies........: NO
Statistical method.................: OES
Maximum assembly length............: 999999
Minimum assembly length............: 50
p-value............................: 0.010000
Quality score threshold (trimming).: 0
Minimum read size after trimming...: 1
Maximal ratio of uncalled bases....: 1.000000
Minimum overlap....................: 10
Scoring method.....................: Scaled score
Threads............................: 8

Allo

 ____  _____    _    ____ 
|  _ \| ____|  / \  |  _ \
| |_) |  _|   / _ \ | |_) |
|  __/| |___ / ___ \|  _ <
|_|   |_____/_/   \_\_| \_\

PEAR v0.9.11 [Nov 5, 2017]

Citation - PEAR: a fast and accurate Illumina Paired-End reAd mergeR
Zhang et al (2014) Bioinformatics 30(5): 614-620 | doi:10.1093/bioinformatics/btt593

Forward reads file.................: /code/sequences/3228.sickle.trimmed.R1.fastq.gz
Reverse reads file.................: /code/sequences/3228.sickle.trimmed.R2.fastq.gz
PHRED..............................: 33
Using empirical frequencies........: NO
Statistical method.................: OES
Maximum assembly length............: 999999
Minimum assembly length............: 50
p-value............................: 0.010000
Quality score threshold (trimming).: 0
Minimum read size after trimming...: 1
Maximal ratio of uncalled bases....: 1.000000
Minimum overlap....................: 10
Scoring method.....................: Scaled score
Threads............................: 8

Allo

Histogram /code/sequences/readyForQiime.allsamples.fasta.histogram.svg finished
Histogram /code/sequences/2618.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2381.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2706.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2788.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/3125.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2903.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/3154.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/3141.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2881.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2426.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2684.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2876.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2549.passedQC.fastq.histogram.svg finished
Histogram /code/sequenc

### 3. Generate sequence counts file used in final step to produce reports

In [5]:
%%time
!for file in sequences/*.passedQC.fastq; \
do \
  awk 'NR%4==2{sum+=1}END{print FILENAME,sum}' $file >> sequences/diatomSequenceCounts.txt; \
done

CPU times: user 800 ms, sys: 150 ms, total: 950 ms
Wall time: 48.6 s


### 4. Assign similar sequences to OTUs using user-defined similarity threshold
#### Default clustering algorithm is UCLUST, O(m\*n), which is a greedy algorithm and is dependent on ordering of sequences in readyForQiime.allsamples.fasta file.

In [ ]:
%%time
!pick_otus.py -i sequences/readyForQiime.allsamples.fasta -o sequences/picked_otus_97

/miniconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


###  5. Pick a representative set of sequences. For each OTU, one sequence will be used in subsequent analysis

In [ ]:
%%time
!pick_rep_set.py -i sequences/picked_otus_97/readyForQiime.allsamples_otus.txt \
  -f sequences/readyForQiime.allsamples.fasta \
  -o sequences/repset.fasta

### 6. Query BLAST database with OTU representatives

In [ ]:
%%time
!blastn -db diatoms -query sequences/repset.fasta \
  -out sequences/repset.diatoms.blastn \
  -task blastn -max_target_seqs 1 -num_threads 8 -outfmt 6 -evalue 0.01

In [ ]:
!mkdir sequences/assigned_taxonomy

### 7. Create taxonomy assignments from BLAST outputs

In [ ]:
%%time
!python ./create_taxonomy_assignments_from_blast.py --taxonomy diatoms.taxonomy.FINAL2017.txt \
  --percid 95.0 --blast sequences/repset.diatoms.blastn --output sequences/assigned_taxonomy/repset.taxonomy.txt 

### 8. Report how often an OTU is found in each sample and add the taxonomic predictions for each OTU

In [ ]:
%%time
!make_otu_table.py -i sequences/picked_otus_97/readyForQiime.allsamples_otus.txt \
  -t sequences/assigned_taxonomy/repset.taxonomy.txt \
  -o sequences/otu_table.biom

### 9. Filter the OTU table based on taxonomic metadata excluding specific taxa

In [ ]:
%%time
!filter_taxa_from_otu_table.py -i sequences/otu_table.biom \
  -o sequences/otu_table.diatomsonly.biom \
  -n MARINE,NOT_DIATOM,Yellow_green_Algae,None

### 10. Sort OTU table by sample id

In [ ]:
%%time
!sort_otu_table.py -i sequences/otu_table.diatomsonly.biom \
  -o sequences/otu_table.diatomsonly.biom

### 11. Provide summary information of the representation of taxonomic groups within each sample

In [ ]:
%%time
!summarize_taxa.py -L 1 \
  -i sequences/otu_table.diatomsonly.biom \
  -o sequences/visualised_taxonomy -a

### 12. Produce Diatom reports

In [ ]:
%%time
!python ./produceDiatomReports.py --folder sequences --lookup lookuptable.txt

### 13. Inspect producted Diatom reports

In [ ]:
import pandas as pd

In [ ]:
pd.read_csv('sequences/Abundances.fail.csv')

In [ ]:
pd.read_csv('sequences/Abundances.pass.csv')